In [379]:
import pandas as pd
import numpy as np
from pybnesian import hc, CLGNetworkType, SemiparametricBNType, SemiparametricBN, MMHC, RCoT, MutualInformation, KMutualInformation, OperatorSet, Score, OperatorPool, ChangeNodeTypeSet, ArcOperatorSet, HoldoutLikelihood, BIC, BGe, CVLikelihood, CKDE, HCKDE, ValidatedLikelihood
#from drawdata import draw_scatter
import matplotlib.pyplot as plt

from bayesace.utils import *
from bayesace.algorithms.bayesace import BayesACE
from bayesace.algorithms.face import FACE

from sklearn.preprocessing import StandardScaler
import openml as oml

import time


In [97]:
df = oml.datasets.get_dataset(44091).get_data()[0]


/tmp/ipykernel_22436/1030542227.py:1: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  df = oml.datasets.get_dataset(44091).get_data()[0]


In [98]:
df["class"] = df["quality"].astype('string').astype('category')
df = df.drop("quality", axis=1)

In [309]:
naive = SemiparametricBN(df.columns)

<bound method PyCapsule.arcs of <pybnesian.SemiparametricBN object at 0x7f1966784970>>

In [113]:
for i in [i for i in df.columns if i != "class"] :
    naive.add_arc("class", i)

In [171]:
df_train = df.head(2000)
df_test = df.tail(500)

In [346]:
test = MutualInformation(df_train, True)
#df = pd.read_csv("toy-3class.csv")
#df["class"] = df["z"].astype('category')
#df["z"] = df["z"].astype('category')
#df = df.drop("z", axis = 1 )

feature_columns = [i for i in df.columns if i != "class"]
df[feature_columns] = StandardScaler().fit_transform(df[feature_columns].values)



learned = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "bic")
learned.fit(df_train)

est = MMHC()
learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType()) #, score = "cv-lik"
learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "holdout-lik", num_folds = 10)
learned_kde.fit(df_train)



In [347]:
for i in df.columns :
    print(i)
    print(learned_kde.cpd(i))
    print()

fixed acidity
[CKDE] P(fixed acidity | chlorides, residual sugar, alcohol, sulphates, total sulfur dioxide) = CKDE with 2000 instances

volatile acidity
[HCKDE] P(volatile acidity | chlorides, sulphates, pH, class, alcohol, density, total sulfur dioxide)
+-------+--------------------------------------------------------------------------------------------------------------------------+
|       |                   volatile acidity | chlorides, sulphates, pH, alcohol, density, total sulfur dioxide                    |
+-------+--------------------------------------------------------------------------------------------------------------------------+
| class |                                                                                                                          |
+-------+--------------------------------------------------------------------------------------------------------------------------+
| False | [CKDE] P(volatile acidity | chlorides, sulphates, pH, alcohol, density

In [348]:
len([i for i in learned_kde.arcs() if "class" in i])

5

In [380]:
labels = np.zeros(shape = (len(df_test.index)))
labels[df_test["class"] == "True"] = 1

sum_sum = []
accuracy = []
ll = []
num_arcs = []
times = []
for i in range(10) :
    t0 = time.time()
    learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",np.mean(accuracy))
print("LL:",np.mean(ll))
print("KDE%:",np.mean(sum_sum))
print("Num arcs:",np.mean(sum_sum))
print("Time:",np.mean(times))
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
times = []
for i in range(10) :
    t0 = time.time()
    learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "cv-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with cv-lik")
print("MSE:",np.mean(accuracy))
print("LL:",np.mean(ll))
print("KDE%:",np.mean(sum_sum))
print("Num arcs:",np.mean(sum_sum))
print("Time:",np.mean(times))
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
times = []
for i in range(10) :
    t0 = time.time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "holdout-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = ValidatedLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("MMHC with val-lik")
print("MSE:",np.mean(accuracy))
print("LL:",np.mean(ll))
print("KDE%:",np.mean(sum_sum))
print("Num arcs:",np.mean(sum_sum))
print("Time:",np.mean(times))
print("----------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
times = []
for i in range(10) :
    t0 = time.time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "holdout-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("MMHC with cv-lik")
print("MSE:",np.mean(accuracy))
print("LL:",np.mean(ll))
print("Num arcs:",np.mean(sum_sum))
print("KDE%:",np.mean(sum_sum))
print("Time:",np.mean(times))


Hill climbing with val-lik
MSE: 0.2832163635855585
LL: -43.79869823891915
KDE%: 66.0
Num arcs: 66.0
Time: 13.86982524394989
--------
Hill climbing with cv-lik
MSE: 0.4306710031021491
LL: -27.717231864352186
KDE%: 100.0
Num arcs: 100.0
Time: 128.51258292198182
--------
MMHC with val-lik
MSE: 0.39567284998386454
LL: -49.89193557559115
KDE%: 49.0
Num arcs: 49.0
Time: 5.205733513832092
----------
MMHC with cv-lik
MSE: 0.34617343234381276
LL: -40.78854972385176
Num arcs: 94.0
KDE%: 94.0
Time: 20.759317064285277


Hill climbing with val-lik
MSE: 0.2832163635855585
LL: -43.79869823891915
KDE%: 66.0
Num arcs: 66.0
Time: 13.86982524394989
--------
Hill climbing with cv-lik
MSE: 0.4306710031021491
LL: -27.717231864352186
KDE%: 100.0
Num arcs: 100.0
Time: 128.51258292198182
--------
MMHC with val-lik
MSE: 0.39567284998386454
LL: -49.89193557559115
KDE%: 49.0
Num arcs: 49.0
Time: 5.205733513832092
----------
MMHC with cv-lik
MSE: 0.34617343234381276
LL: -40.78854972385176
Num arcs: 94.0
KDE%: 94.0
Time: 20.759317064285277

In [381]:
labels = np.zeros(shape = (len(df_test.index)))
labels[df_test["class"] == "True"] = 1

sum_sum = []
accuracy = []
ll = []
num_arcs = []
times = []
for i in range(10) :
    t0 = time.process_time()
    learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
print("--------")


sum_sum = []
accuracy = []
ll = []
num_arcs = []
times = []
for i in range(10) :
    t0 = time.process_time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "holdout-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("MMHC with cv-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))

Hill climbing with val-lik
MSE: 0.39 +- 0.03
LL: -42.42 +- 7.41
Num arcs: 24.0 +- 13.47
KDE%: 63.0 +- 16.16
Time: 19.96 +- 13.33
--------
MMHC with cv-lik
MSE: 0.36 +- 0.01
LL: -40.78 +- 0.42
Num arcs: 26.5 +- 0.5
KDE%: 92.0 +- 4.0
Time: 18.5 +- 2.48


In [396]:
df = df.sample(frac = 1)
df_train = df.head(2000)
df_test = df.tail(500)

labels = np.zeros(shape = (len(df_test.index)))
labels[df_test["class"] == "True"] = 1

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(50) :
    t0 = time.process_time()
    learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_hc = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(50) :
    t0 = time.process_time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = ValidatedLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_mmhc_val = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(50) :
    t0 = time.process_time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_mmhc_cv = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

Hill climbing with val-lik
MSE: 0.22 +- 0.03
LL: -46.31 +- 10.43
Num arcs: 16.46 +- 11.31
Class arcs: 1.22 +- 1.27
KDE%: 54.6 +- 30.54
Time: 12.21 +- 9.09
--------
Hill climbing with val-lik
MSE: 0.22 +- 0.03
LL: -49.43 +- 8.52
Num arcs: 12.16 +- 8.21
Class arcs: 1.14 +- 1.4
KDE%: 53.2 +- 30.43
Time: 5.79 +- 4.26
--------
Hill climbing with val-lik
MSE: 0.16 +- 0.0
LL: -39.15 +- 0.29
Num arcs: 27.36 +- 0.84
Class arcs: 6.14 +- 0.45
KDE%: 99.4 +- 3.1
Time: 18.89 +- 2.76
--------


In [414]:
analysis_hc

,MSE,logl,n_arcs,class_arcs,kdes,ptime
0,0.244384,-43.018614,17.0,1.0,60.0,10.833885
1,0.177075,-34.551579,36.0,4.0,90.0,27.276015
2,0.197943,-43.079298,17.0,1.0,60.0,11.546251
3,0.216749,-47.534548,11.0,1.0,60.0,6.697710
4,0.194404,-36.607270,30.0,3.0,90.0,21.945917
5,0.216749,-44.772460,13.0,1.0,60.0,9.123855
6,0.244384,-42.870112,17.0,1.0,60.0,10.537385
7,0.250076,-61.843395,1.0,0.0,10.0,0.383535
8,0.216749,-42.178945,17.0,1.0,70.0,12.621452
9,0.250076,-44.980750,13.0,0.0,50.0,9.330226


In [412]:
analysis_hc[analysis_hc["class_arcs"]==4]

,MSE,logl,n_arcs,class_arcs,kdes,ptime
1,0.177075,-34.551579,36.0,4.0,90.0,27.276015
10,0.182381,-35.097220,35.0,4.0,90.0,25.809887
12,0.179099,-36.307605,30.0,4.0,90.0,25.446741
42,0.173735,-36.766162,30.0,4.0,80.0,24.344421


In [416]:
from scipy.stats import kstest
from scipy.stats import lognorm

kstest(df["density"], 'norm')


KstestResult(statistic=0.06431654899817901, pvalue=1.253587757507702e-09, statistic_location=-0.5362215669681757, statistic_sign=1)

In [372]:
sum = 0
for i in learned_kde.nodes()[:-2] :    
    if isinstance(learned_kde.cpd(i), CKDE) or isinstance(learned_kde.cpd(i), HCKDE) :
        sum = sum + 1
print(sum/10 * 100)

100.0


In [356]:
me_hc_h = 0.564
me_mmhc_h = 0.52
me_hc_cv = 0.59
me_mmhc_cv = 0.46

0.4489779380314274

In [394]:
len([i for i in learned_kde.arcs() if "class" in i])

[('class', 'citric acid'), ('class', 'alcohol')]

In [54]:
accuracy(instance.drop("class", axis = 1), "True", learned_kde)/

array([0.5])

In [60]:
instance = df.iloc[[0]]
instance["class"] = pd.Categorical(["False"], categories=learned_kde.cpd("class").variable_values())
(math.e ** learned.logl(instance)) / likelihood(instance.drop("class", axis = 1), learned_kde)

/tmp/ipykernel_22436/3126163979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instance["class"] = pd.Categorical(["False"], categories=learned_kde.cpd("class").variable_values())


array([0.03584753])

In [ ]:
n_vertex = 0
x_og = df.iloc[[0]]
bayesace = BayesACE(bayesian_network=learned_kde, features= df.columns[:-1], n_vertex=n_vertex, chunks = 100, penalty=1, pop_size=50, likelihood_threshold=0.0, accuracy_threshold= 0.5, generations=10)
res = bayesace.run(x_og)

In [ ]:
res.counterfactual

In [ ]:
df.iloc[0]

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.plot(df_vertex.x,df_vertex.y,color = "red")
plt.show()

In [ ]:
n_vertex = 1
x_og = df.iloc[[0]]
face = FACE(bayesian_network=learned_kde, features= df.columns[:-1], dataset=df.drop("class", axis = 1), graph_type = "epsilon", chunks = 10, penalty=1, distance_threshold=0.75, accuracy_threshold= 0.5, verbose = False)
res = face.run(x_og)

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.plot(df_vertex.x,df_vertex.y,color = "red")
plt.show()